In [ ]:
import ale_py
import gymnasium as gym
import numpy as np
from gymnasium.wrappers import (
    AtariPreprocessing,
    FrameStackObservation,
    TimeLimit,
    TransformObservation,
)

from src.config import build_cfg
from src.train import train_dql_agent

%load_ext autoreload
%autoreload 2


cfg = build_cfg(
    "src/configs/atari.yaml",
)

gym.register_envs(ale_py)
env = gym.make("ALE/Breakout-v5", frameskip=1, render_mode="rgb_array")
env = AtariPreprocessing(env)
env = FrameStackObservation(env, stack_size=4)


def swap_channels(obs):
    # obs ha shape (4, 84, 84)
    return np.transpose(obs, (1, 2, 0))  # → (84, 84, 4)


old_space = env.observation_space
new_space = gym.spaces.Box(
    low=old_space.low.min(),
    high=old_space.high.max(),
    shape=(old_space.shape[1], old_space.shape[2], old_space.shape[0]),
    dtype=old_space.dtype,
)

env = TransformObservation(env, swap_channels, new_space)
env = TimeLimit(env, max_episode_steps=cfg.max_steps_per_episode)
state_dim = env.observation_space.shape
train_dql_agent(cfg, env, state_dim)